In [1]:
#Install
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
Using cached flatbuffers-24.3.25-py2.py3-none-any.whl (26 kB)
  Using cached pandas-2.2.2-cp310-cp310-win_amd64.whl.metadata (19 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/61.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/61.4 kB ? eta -:--:--
   ------ --------------------------------- 10.2/61.4 kB ? eta -:--:--
   -------------------- ------------------- 30.7/61.4 kB 325.1 kB/s eta 0:00:01
   -------------------- ------------------- 30.7/61.4 kB 325.1 kB/s eta 0:00:01
   -------------------- ------------------- 30.7/61.4 kB 325.1 kB/s eta 0:00:01
   -------------------- ------------------- 30.7/61.4 kB 325.1 kB/s eta 0:00:01
   -------------------------- ------------- 41.0/61.4 kB 131.3 

In [2]:
#Import Dependencies
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

In [3]:
#Setup the Open-Meteo API Client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [4]:
#Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": [36.154, 40.7128, 34.0549],
	"longitude": [95.9928, 74.006, 118.2426],
	"start_date": "2014-01-01",
	"end_date": "2023-12-31",
	"hourly": "relative_humidity_2m",
	"daily": ["temperature_2m_max", "temperature_2m_min", "precipitation_sum"],
	"temperature_unit": "fahrenheit",
	"precipitation_unit": "inch",
	"timezone": "America/Chicago"
}
responses = openmeteo.weather_api(url, params=params)

In [5]:
# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

Coordinates 36.16871643066406°N 96.07791900634766°E
Elevation 4246.0 m asl
Timezone b'America/Chicago' b'CDT'
Timezone difference to GMT+0 -18000 s


In [6]:
# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_relative_humidity_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

Coordinates 36.16871643066406°N 96.07791900634766°E
Elevation 4246.0 m asl
Timezone b'America/Chicago' b'CDT'
Timezone difference to GMT+0 -18000 s
                           date  relative_humidity_2m
0     2014-01-01 05:00:00+00:00             59.558182
1     2014-01-01 06:00:00+00:00             55.147812
2     2014-01-01 07:00:00+00:00             56.722488
3     2014-01-01 08:00:00+00:00             61.792240
4     2014-01-01 09:00:00+00:00             64.751877
...                         ...                   ...
87643 2024-01-01 00:00:00+00:00             41.523659
87644 2024-01-01 01:00:00+00:00             44.350803
87645 2024-01-01 02:00:00+00:00             39.358746
87646 2024-01-01 03:00:00+00:00             35.514065
87647 2024-01-01 04:00:00+00:00             37.143208

[87648 rows x 2 columns]


In [7]:
# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(2).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["precipitation_sum"] = daily_precipitation_sum

daily_dataframe = pd.DataFrame(data = daily_data)
print(daily_dataframe)

                          date  temperature_2m_max  temperature_2m_min  \
0    2014-01-01 05:00:00+00:00           19.540400           -6.199600   
1    2014-01-02 05:00:00+00:00            9.550400          -24.199596   
2    2014-01-03 05:00:00+00:00            6.940401          -24.559601   
3    2014-01-04 05:00:00+00:00           18.820400          -14.029602   
4    2014-01-05 05:00:00+00:00           14.050400          -12.409599   
...                        ...                 ...                 ...   
3647 2023-12-27 05:00:00+00:00           24.994400           17.704399   
3648 2023-12-28 05:00:00+00:00           19.594400            4.834400   
3649 2023-12-29 05:00:00+00:00           21.214401            9.064398   
3650 2023-12-30 05:00:00+00:00           23.554399           17.434399   
3651 2023-12-31 05:00:00+00:00           23.914400            1.774403   

      precipitation_sum  
0                   0.0  
1                   0.0  
2                   0.0  
3      